In [4]:
%load_ext autoreload
%autoreload 2
import os
import copy
import pickle
from functools import lru_cache
import pandas as pd
import copy
import time
import sys
from statistics import mean

import numpy as np
import scipy
from tqdm import tqdm

from gensim.models import Word2Vec
from gensim import utils
from gensim.corpora import Dictionary
from gensim.models.lsimodel import LsiModel
from sklearn.preprocessing import MinMaxScaler

import sys
sys.path.append('../src')
from models import calculate_association_metric_for_target_word, word_set_to_mtx, \
    get_1storder_association_metric_no_cache, get_1storder_association_metric, \
    get_expSG_1storder_relation_no_cache_NEW, get_expSG_1storder_relation_no_cache_NEW_ALLWORDS


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
we_model_name = "sg_dim300_min100_win5"
we_vector_size = 300
we_model_dir = '../data/external/wiki-english/wiki-english-20171001/%s' % we_model_name

In [6]:
we_model = Word2Vec.load(we_model_dir+'/model.gensim')

print ('loading done!')

loading done!


In [4]:
#loading E_ctx_vec
with open(we_model_dir+'/E_ctx_vec.pkl', 'rb') as fr:
    E_ctx_vec = pickle.load(fr)
with open(we_model_dir+'/E_wrd_vec.pkl', 'rb') as fr:
    E_wrd_vec = pickle.load(fr)

# Calculating *d*

In [5]:
# WEAT 1
X_terms = ['aster', 'clover', 'hyacinth', 'marigold', 'poppy', 'azalea', 
           'crocus', 'iris', 'orchid', 'rose', 'bluebell', 'daffodil','lilac',
           'pansy','tulip','buttercup','daisy','lily','penny','violet','carnation',
           'magnolia','petunia','zinnia']
Y_terms = ['caterpillar','flea','locust','spider','bedbug','centipede','fly',
          'maggot','tarantula','bee','cockroach','gnat','mosquito','termite','beetle',
          'cricket','hornet','moth','wasp','blackfly','dragonfly','horsefly','roach',
          'weevil']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
           'loyal','pleasure','diamond','gentle','honest','lucky','rainbow','diploma',
           'gift','honor','miracle','sunrise','family','happy','laugher','paradise',
           'vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy',
          'divorce','jail','poverty','ugly','cancer','kill','rotten','vomit','agony',
          'prison']
# Removed words:
# X: gladiola (not found)
# Y: ant (random)

In [7]:
# WEAT 2
X_terms = ['bagpipe','cello','guitar','lute','trombone','banjo','clarinet','harmonica',
           'mandolin','trumpet','bassoon','drum','harp','oboe','tuba','bell','fiddle',
           'harpsichord','piano','viola','bongo','flute','horn','saxophone']
Y_terms = ['arrow','club','gun','missile','spear','axe','dagger','harpoon','pistol',
          'sword','blade','dynamite','hatchet','rifle','tank','bomb','firearm',
          'knife','shotgun','teargas','cannon','grenade','mace','slingshot','whip']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
           'loyal','pleasure','diamond','gentle','honest','lucky','rainbow','diploma',
           'gift','honor','miracle','sunrise','family','happy','laugher','paradise',
           'vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy',
          'divorce','jail','poverty','ugly','cancer','kill','rotten','vomit','agony',
          'prison']

In [7]:
# WEAT 3
X_terms = ['roger','alan','frank','ian','justin',
          'ryan','andrew','fred','jack','matthew','stephen','brad','greg','jed',
          'paul','todd','brandon','hank','jonathan','peter','wilbur','amanda',
          'courtney','heather','melanie','sara','amber','crystal','katie',
          'meredith','shannon','betsy','donna','kristin','nancy','stephanie',
          'bobbie-sue','ellen','lauren','peggy','sue-ellen','colleen','emily',
          'megan','rachel','wendy']
Y_terms = ['alonzo','jamel','lerone','theo','alphonse','jerome','leroy',
           'torrance','darnell','lamar','lionel','rashaun','tyree','deion',
          'lamont','malik','terrence','tyrone','lavon','terryl',
          'wardell','aiesha','lashelle','nichelle','shereen','temeka','ebony',
          'latisha','shaniqua','tameisha','teretha','jasmine','laronya','shanise',
          'tanisha','tia','lakisha','latoya','sharise','tashika','yolanda',
          'lashandra','malika','shavonn','tawanda','yvette']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
          'loyal','pleasure','diamond','gentle','honest','lucky','rainbow',
          'diploma','gift','honor','miracle','sunrise','family','happy','laughter',
          'paradise','vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy','bomb',
          'divorce','jail','poverty','ugly','cancer','evil','kill','rotten','vomit']
print ("Missing x terms: ", [x for x in X_terms if x not in we_model.wv])
print('missing y terms: ', [x for x in Y_terms if x not in we_model.wv])
# Removed terms
# X: adam (random), chip (random), harry (random), josh (random)
# Y: percell (not found), rasaan (not found), everol (not found), marcelius (not found)

Missing x terms:  ['bobbie-sue', 'sue-ellen']
missing y terms:  ['terryl', 'aiesha', 'lashelle', 'temeka', 'tameisha', 'teretha', 'laronya', 'shanise', 'lakisha', 'sharise', 'tashika', 'lashandra', 'shavonn']


In [8]:
# WEAT 4
X_terms = ['Brad', 'Brendan', 'Geoffrey', 'Greg', 'Brett', 'Jay', 'Matthew', 'Neil',
'Todd', 'Allison', 'Anne', 'Carrie', 'Emily', 'Jill', 'Laurie', 'Kristen', 'Meredith',
           'Sarah']
Y_terms = ['Darnell', 'Hakim', 'Jermaine', 'Kareem', 'Jamal', 'Leroy', 'Rasheed',
'Tremayne', 'Tyrone', 'Aisha', 'Ebony', 'Keisha', 'Kenya', 'Latonya', 'Latoya',
           'Tamika', 'Tanisha']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
           'loyal','pleasure','diamond','gentle','honest','lucky','rainbow','diploma',
           'gift','honor','miracle','sunrise','family','happy','laugher','paradise',
           'vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy',
          'bomb','divorce','jail','poverty','ugly','cancer','evil','kill',
          'rotten','vomit']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]


In [9]:
# WEAT 5
X_terms = ['Brad', 'Brendan', 'Geoffrey', 'Greg', 'Brett', 'Jay', 'Matthew', 'Neil',
'Todd', 'Allison', 'Anne', 'Carrie', 'Emily', 'Jill', 'Laurie', 'Kristen', 'Meredith',
           'Sarah']
Y_terms = ['Darnell', 'Hakim', 'Jermaine', 'Kareem', 'Jamal', 'Leroy', 'Rasheed',
'Tremayne', 'Tyrone', 'Aisha', 'Ebony', 'Keisha', 'Kenya', 'Latonya', 'Latoya',
           'Tamika', 'Tanisha']
A_terms = ['joy', 'love', 'peace', 'wonderful', 'pleasure', 'friend', 'laughter', 'happy']
B_terms = ['agony', 'terrible', 'horrible', 'nasty', 'evil', 'war', 'awful', 'failure']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]
# Removed terms: "Lakisha"

In [5]:
# WEAT 6
X_terms = ['John', 'Paul','Mike','Kevin','Steve','Greg','Jeff','Bill']
Y_terms = ['Amy','Joan','Lisa','Sarah','Diana','Kate','Ann','Donna']
A_terms = ['executive','management','professional','corporation',
               'salary','office','business','career']
B_terms = ['home','parents','children','family',
               'cousins','marriage','wedding','relatives']
[X_terms, Y_terms, A_tersm, B_terms] = [tuple([str.lower(term) for term in terms]) for terms in [X_terms, Y_terms, A_terms, B_terms]]

In [11]:
# WEAT 7 using Google News word2vec model
X_terms = ['math','algebra','geometry','calculus',
             'equations','computation','numbers','addition']
Y_terms = ['poetry','art','dance','literature',
             'novel','symphony','drama','sculpture']
A_terms = ['male','man','boy','brother',
              'he','him','his','son']
B_terms = ['female','woman','girl','sister',
               'she','her','hers','daughter']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]

In [12]:
# Weat 8
A_terms = ['brother','father','uncle','grandfather',
          'son','he','his','him']
B_terms = ['sister','mother','aunt','grandmother',
          'daughter','she','hers','her']
X_terms = ['science','technology','physics','chemistry',
          'Einstein','NASA','experiment','astronomy']
Y_terms = ['poetry','art','Shakespeare','dance',
          'literature','novel','symphony','drama']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]

In [13]:
# WEAT 9
X_terms = ['sad','hopeless','gloomy','tearful','miserable','depressed']
Y_terms = ['sick','illness','influenza','disease','virus','cancer']
A_terms = ['impermanent','unstable','variable','fleeting','short','brief',
          'occasional']
B_terms = ['stable','always','constant','persistent','chronic','prolonged','forever']

In [14]:
# Weat 10
X_terms = ['Tiffany', 'Michelle', 'Cindy', 'Kristy', 'Brad', 'Eric', 'Joey', 'Billy']
Y_terms = ['Ethel', 'Bernice', 'Gertrude', 'Agnes', 'Cecil', 'Wilbert', 'Mortimer', 'Edgar']
A_terms = ['joy', 'love', 'peace', 'wonderful', 'pleasure', 'friend', 'laughter', 'happy']
B_terms = ['agony', 'terrible', 'horrible', 'nasty', 'evil', 'war', 'awful', 'failure']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]

# Generating List of Association Metrics

# Second-Order

In [15]:
def get_association_metrics_for_term_list(wv_obj, target_terms, A_terms, B_terms):
    target_term_mtx = word_set_to_mtx(wv_obj, target_terms)
    A_mtx = word_set_to_mtx(wv_obj, A_terms)
    B_mtx = word_set_to_mtx(wv_obj, B_terms)
    return np.apply_along_axis(lambda x_vec: calculate_association_metric_for_target_word(x_vec, A_mtx, B_mtx), 1, target_term_mtx)

In [16]:
get_association_metrics_for_term_list(we_model, X_terms, A_terms, B_terms)

array([0.05075578, 0.07742482, 0.04195428, 0.04895789, 0.01750761,
       0.05121625, 0.03581539, 0.01980541])

# After Navid's Fast Functions

In [8]:
get_expSG_1storder_relation_no_cache_NEW_ALLWORDS(A_terms, we_model)

array([[0.04645412, 0.09929683, 0.08176807, ..., 0.15134957, 0.19802235,
        0.0905898 ],
       [0.04178686, 0.1322454 , 0.08583836, ..., 0.04314406, 0.12817161,
        0.11693659],
       [0.03590716, 0.15498559, 0.12907532, ..., 0.01956923, 0.10972705,
        0.11150997],
       ...,
       [0.00033812, 0.00766713, 0.00406804, ..., 0.0017368 , 0.01426721,
        0.01477553],
       [0.00038508, 0.00992056, 0.00371194, ..., 0.00025123, 0.02094633,
        0.00301536],
       [0.00029049, 0.00210694, 0.00248915, ..., 0.00067894, 0.02516706,
        0.00291943]], dtype=float32)

# First-Order

In [6]:
def get_1storder_association_metrics_for_term_list(wv_obj, target_terms, A_terms, B_terms, E_ctx_vec, E_wrd_vec):
    x_associations = np.array([])
    [A_terms, B_terms, E_ctx_vec_tup, E_wrd_vec_tup] = [tuple(i) for i in [A_terms, B_terms, E_ctx_vec, E_wrd_vec]]
    for x in tqdm(target_terms):
        x_association = get_1storder_association_metric(x, A_terms, B_terms, we_model, E_ctx_vec_tup, E_wrd_vec_tup)
        x_associations = np.append(x_associations, x_association)
    return x_associations
get_1storder_association_metrics_for_term_list(we_model, X_terms, A_terms, B_terms, E_ctx_vec, E_wrd_vec)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  4.50it/s]


array([-0.51408482, -0.29574019,  0.15860224, -0.07316208,  0.20548844,
       -0.29844368,  0.05948937,  0.34631753])

In [7]:
def get_1storder_association_metrics_for_term_list_no_cache(wv_obj, target_terms, A_terms, B_terms, E_ctx_vec, E_wrd_vec):
    x_associations = np.array([])
    for x in tqdm(target_terms):
        x_association = get_1storder_association_metric_no_cache(x, A_terms, B_terms, we_model, E_ctx_vec, E_wrd_vec)
        x_associations = np.append(x_associations, x_association)
    return x_associations
get_1storder_association_metrics_for_term_list_no_cache(we_model, X_terms*10, A_terms, B_terms, E_ctx_vec, E_wrd_vec)

100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:06<00:00, 12.29it/s]


array([-0.51408482, -0.29574019,  0.15860224, -0.07316208,  0.20548844,
       -0.29844368,  0.05948937,  0.34631753, -0.51408482, -0.29574019,
        0.15860224, -0.07316208,  0.20548844, -0.29844368,  0.05948937,
        0.34631753, -0.51408482, -0.29574019,  0.15860224, -0.07316208,
        0.20548844, -0.29844368,  0.05948937,  0.34631753, -0.51408482,
       -0.29574019,  0.15860224, -0.07316208,  0.20548844, -0.29844368,
        0.05948937,  0.34631753, -0.51408482, -0.29574019,  0.15860224,
       -0.07316208,  0.20548844, -0.29844368,  0.05948937,  0.34631753,
       -0.51408482, -0.29574019,  0.15860224, -0.07316208,  0.20548844,
       -0.29844368,  0.05948937,  0.34631753, -0.51408482, -0.29574019,
        0.15860224, -0.07316208,  0.20548844, -0.29844368,  0.05948937,
        0.34631753, -0.51408482, -0.29574019,  0.15860224, -0.07316208,
        0.20548844, -0.29844368,  0.05948937,  0.34631753, -0.51408482,
       -0.29574019,  0.15860224, -0.07316208,  0.20548844, -0.29

In [8]:
def np_get_1storder_association_metrics_for_term_list_no_cache(wv_obj, target_terms, A_terms, B_terms, E_ctx_vec, E_wrd_vec):
    word_array = np.array(target_terms)
    word_array = word_array.reshape(word_array.shape[0],1)
    return np.apply_along_axis(lambda word_row: get_1storder_association_metric_no_cache(word_row[0], A_terms, B_terms, we_model, E_ctx_vec, E_wrd_vec),
                               1, 
                               word_array
                              )
np_get_1storder_association_metrics_for_term_list_no_cache(we_model, X_terms, A_terms, B_terms, E_ctx_vec, E_wrd_vec)

array([-0.5140848 , -0.2957402 ,  0.15860224, -0.07316208,  0.20548844,
       -0.29844368,  0.05948937,  0.34631753], dtype=float32)

In [9]:
start = time.time()
all_terms = list(we_model.wv.vocab.keys())[:100]
arr = np.array(all_terms)
reshaped = arr.reshape(arr.shape[0],1)
out = np.apply_along_axis(lambda word_row: get_1storder_association_metric_no_cache(word_row[0], A_terms, B_terms, we_model, E_ctx_vec, E_wrd_vec),
                    1,
                    reshaped
                   )
end = time.time()
print(end - start)
print(out)

7.618658781051636
[-0.34742123  0.22945678  0.20701694  0.63986444  0.7723202  -0.3124889
  0.21972203  0.27452683  1.5307587   0.5099771   0.83211786  0.04580712
  0.6653863   1.3520031  -0.44613564 -0.69415826 -0.93914783 -0.8458704
  0.6457273  -1.5933924  -0.507394   -0.00813615 -0.25980848 -0.887969
  0.32272536 -1.3484201  -0.1401447  -0.1868093   0.8077446   0.65310675
  0.13093007  1.1589237   1.1786904   0.19201088  0.84863913 -0.26915395
 -0.26890397 -0.78745955 -0.18705809  0.0551002  -0.58185     0.16298419
 -0.36246085  0.06116647  0.25085807  1.1216156   0.24398136 -0.40291
  2.2966957  -0.31740403  1.5783508   0.17954671  0.52805233  0.24403167
 -0.3412379   0.29118848  1.2576344   0.09002447 -0.6491295  -0.5256914
  1.6953359   1.5536044  -0.5038368  -0.2901739  -0.32822526 -0.53872514
 -0.22055054 -0.48665202 -0.43565488  3.0136778   0.67921126 -0.25621164
 -0.36676967 -0.28988987 -0.34295392 -0.01092619  0.22659934 -0.46584058
  1.2016926  -0.8392527   0.5808321   0.7

## Max for Min-Max

In [22]:
all_terms = list(we_model.wv.vocab.keys())
whole_vocab_association_metrics = get_1storder_association_metrics_for_term_list_no_cache(we_model, all_terms, A_terms, B_terms, E_ctx_vec, E_wrd_vec)

  0%|                                                                           | 176/312425 [00:20<9:51:15,  8.80it/s]

KeyboardInterrupt: 

# Sandbox

In [ ]:
len(we_model.wv.vectors)

In [ ]:
all_terms = list(we_model.wv.vocab.keys())
all_terms[:5]